In [1]:
import pandas as pd
import json

In [2]:
results = pd.read_csv('data/final_results_map.csv', index_col=None)

In [3]:
display(results.head())
display(len(results))
# display(results.dtypes)

,state,district,won,won_pred,rel_won_proba,correct_pred
0,Alabama,District 1,1,1.0,0.996061,True
1,Alabama,District 2,1,1.0,0.991629,True
2,Alabama,District 3,1,1.0,0.878124,True
3,Alabama,District 4,1,1.0,0.993328,True
4,Alabama,District 5,1,1.0,0.945481,True


435

In [23]:
with open('data/map/us_congressional_districts.json') as f:
    map_data = json.load(f)

at_large_dists = ['Alaska', 'Delaware', 'Montana', 'Wyoming', 'Vermont', 'South Dakota', 'North Dakota',
                 'Puerto Rico', 'District of Columbia']
    
i = 0
for geometry in map_data['objects']['us_congressional_districts']['geometries']:
    district = geometry['properties']
    # print(district['STATE'], district['CONG_DIST'])
    # Find the results for this district
    if district["STATE"] in at_large_dists:
        res = results.loc[(results['state'] == district['STATE']) & \
                (results['district'] == 'At-Large')]
    else:
        res = results.loc[(results['state'] == district['STATE']) & \
                (results['district'] == 'District {}'.format(int(district['CONG_DIST'])))]
        
    if len(res) > 0:
        # Update the informations
        props = {
            'STATE': district['STATE'],
            'STATE_FIPS': district['STATE_FIPS'],
            'CONG_DIST': district['CONG_DIST'],
            'CONG_REP': district['CONG_REP'],
            'PARTY_AFF': 'Republican' if res['won'].values[0] == 1 else 'Democrat',
            'Shape_Leng': district['Shape_Leng'],
            'Shape_Area': district['Shape_Area'],
            'alpha': res['rel_won_proba'].values[0]
        }
    else:
        print(district['STATE'], district['CONG_DIST'], 'No result for this district')
        props = {
            'STATE': district['STATE'],
            'STATE_FIPS': district['STATE_FIPS'],
            'CONG_DIST': district['CONG_DIST'],
            'CONG_REP': district['CONG_REP'],
            'PARTY_AFF': 'NaN',
            'Shape_Leng': district['Shape_Leng'],
            'Shape_Area': district['Shape_Area'],
            'alpha': 'NaN'
        }
    
    map_data['objects']['us_congressional_districts']['geometries'][i]['properties'] = props
    
    i = i+1
    
# Save the updated data
with open('data/map/us_house_results_map.json', 'w') as outfile:
    json.dump(map_data, outfile)

Puerto Rico 00 No result for this district
District of Columbia 00 No result for this district
U.S. Virgin Islands 00 No result for this district
Guam 00 No result for this district
Northern Mariana Islands 00 No result for this district
American Samoa 00 No result for this district
